In [18]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Line, Bar, Pie, Page

# 1. 读取 CSV 文件中的销售数据
data = pd.read_csv('../tmp/order.csv', encoding='gbk')

# 2. 对数据进行分组统计
monthly_stats = data.groupby(['月份']).agg({
    '设备编号': 'nunique',  # 计算设备数量
    '总金额（元）': 'sum'  # 计算销售额
}).reset_index()

city_stats = data.groupby(['市']).agg({
    '总金额（元）': 'sum',  # 计算销售总额
    '设备编号': 'nunique'  # 计算设备数量
}).reset_index()

# 创建一个页面
page = Page()

# 3. 使用 Line 图表展示销售额和设备数量随月份的变化趋势
line = Line()
line.add_xaxis(monthly_stats['月份'].astype(str).tolist())
line.add_yaxis("销售额", monthly_stats['总金额（元）'].tolist(), is_smooth=True)
line.add_yaxis("设备数量", monthly_stats['设备编号'].tolist(), is_smooth=True)
line.set_global_opts(title_opts=opts.TitleOpts(title="销售额与设备数量随月份变化趋势"),
                     xaxis_opts=opts.AxisOpts(name="月份"),
                     yaxis_opts=opts.AxisOpts(name="金额与数量"))
page.add(line)

# 4. 使用 Bar 图表展示订单数量和设备数量随月份的变化趋势
order_counts = data.groupby(['月份']).size().reset_index(name='订单数量')

bar1 = Bar()
bar1.add_xaxis(monthly_stats['月份'].astype(str).tolist())
bar1.add_yaxis("订单数量", order_counts['订单数量'].tolist())
bar1.add_yaxis("设备数量", monthly_stats['设备编号'].tolist())
bar1.set_global_opts(title_opts=opts.TitleOpts(title="订单数量与设备数量随月份变化趋势"),
                     xaxis_opts=opts.AxisOpts(name="月份"),
                     yaxis_opts=opts.AxisOpts(name="数量"))
page.add(bar1)

# 5. 使用 Bar 图表展示各城市的自动售货机平均销售总额
city_stats['平均销售总额'] = city_stats['总金额（元）'] / city_stats['设备编号']
bar2 = Bar()
bar2.add_xaxis(city_stats['市'].tolist())
bar2.add_yaxis("平均销售总额", city_stats['平均销售总额'].tolist())
bar2.set_global_opts(title_opts=opts.TitleOpts(title="各城市自动售货机平均销售总额"),
                     xaxis_opts=opts.AxisOpts(name="城市"),
                     yaxis_opts=opts.AxisOpts(name="平均销售总额"))
page.add(bar2)

# 6. 使用 Bar 图表展示销售额排名前 10 和后 10 的商品
top_products = data.groupby(['商品名称'])['总金额（元）'].sum().reset_index()
top_products = top_products.sort_values(by='总金额（元）', ascending=False)
top_10_products = top_products.head(10)
bottom_10_products = top_products.tail(10)

bar3 = Bar()
bar3.add_xaxis(top_10_products['商品名称'].tolist() + bottom_10_products['商品名称'].tolist())
bar3.add_yaxis("销售额", top_10_products['总金额（元）'].tolist() + bottom_10_products['总金额（元）'].tolist())
bar3.set_global_opts(title_opts=opts.TitleOpts(title="销售额排名前 10 和后 10 的商品"),
                     xaxis_opts=opts.AxisOpts(name="商品名称", axislabel_opts=opts.LabelOpts(rotate=30)),
                     yaxis_opts=opts.AxisOpts(name="销售额"))
page.add(bar3)

# 7. 使用 Bar 图表展示销售额排名前 10 和后 10 的设备及其所在城市
top_devices = data.groupby(['设备编号', '市'])['总金额（元）'].sum().reset_index()
top_devices = top_devices.sort_values(by='总金额（元）', ascending=False)
top_10_devices = top_devices.head(10)
bottom_10_devices = top_devices.tail(10)

bar4 = Bar()
bar4.add_xaxis(top_10_devices['设备编号'].astype(str).tolist() + bottom_10_devices['设备编号'].astype(str).tolist())
bar4.add_yaxis("销售额", top_10_devices['总金额（元）'].tolist() + bottom_10_devices['总金额（元）'].tolist())
bar4.set_global_opts(title_opts=opts.TitleOpts(title="销售额排名前 10 和后 10 的设备及其所在城市"),
                     xaxis_opts=opts.AxisOpts(name="设备编号"),
                     yaxis_opts=opts.AxisOpts(name="销售额"))
page.add(bar4)

# 8. 使用 Bar 图表展示销售额小于 100 的设备数量在各城市的分布
low_sales_devices = data[data['总金额（元）'] < 100].groupby(['市'])['设备编号'].nunique().reset_index(name='数量')
bar5 = Bar()
bar5.add_xaxis(low_sales_devices['市'].tolist())
bar5.add_yaxis("数量", low_sales_devices['数量'].tolist())
bar5.set_global_opts(title_opts=opts.TitleOpts(title="销售额小于 100 的设备数量在各城市的分布"),
                     xaxis_opts=opts.AxisOpts(name="城市"),
                     yaxis_opts=opts.AxisOpts(name="数量"))
page.add(bar5)

# 9. 使用 Pie 图表展示订单支付方式和各消费时段的订单用户占比
payment_stats = data.groupby(['支付状态'])['订单编号'].count().reset_index()
time_period_stats = data.groupby(['下单时间段'])['订单编号'].count().reset_index()

pie1 = Pie()
pie1.add("", [list(z) for z in zip(payment_stats['支付状态'].tolist(), payment_stats['订单编号'].tolist())])
pie1.set_global_opts(title_opts=opts.TitleOpts(title="订单支付方式占比"))
pie1.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
page.add(pie1)

pie2 = Pie()
pie2.add("", [list(z) for z in zip(time_period_stats['下单时间段'].tolist(), time_period_stats['订单编号'].tolist())])
pie2.set_global_opts(title_opts=opts.TitleOpts(title="各消费时段的订单用户占比"))
pie2.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
page.add(pie2)

# 渲染页面到一个 HTML 文件
page.render("all_charts.html")


TypeError: cannot convert dictionary update sequence element #0 to a sequence